### contexto:

la columna 'address' conntiene info importante sobre como tratar los jsons de estados. ya que contiene informacion de direccion y ciudades.

si se normaliza esta columna en una tabla aparte que contenga: la ciudad la direccion y codigo postal, mas la clave unica gmap_id, podemos agrupar por estados con sus ciudades.

con un eda sobre las categorias de los negocios podemos decidir que estados usar y cuales no, basandonos en datos y no en suposiciones.

informacion: 

aqui solo hare codigo, no voy a documentar, si tienen dudas se lo explico en una reunion que hagamos.

In [32]:
import pandas as pd

sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')
sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2998428 entries, 0 to 3025010
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   address         object 
 2   gmap_id         object 
 3   description     object 
 4   latitude        float64
 5   longitude       float64
 6   avg_rating      float64
 7   num_of_reviews  int64  
 8   price           object 
 9   state           object 
 10  url             object 
 11  monday          object 
 12  tuesday         object 
 13  wednesday       object 
 14  thursday        object 
 15  friday          object 
 16  saturday        object 
 17  sunday          object 
dtypes: float64(3), int64(1), object(14)
memory usage: 434.6+ MB


In [33]:
address = sitios[['gmap_id','address', 'name']]
address.head()

,gmap_id,address,name
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",San Soo Dang
3,0x80c2c89923b27a41:0x32041559418d447,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",Nova Fabrics
4,0x80c2c632f933b073:0xc31785961fe826a6,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",Nobel Textile Co


In [34]:
# Separar la columna 'address' en nuevas columnas
address_split = address['address'].str.split(',', n=3, expand=True)

# Cambiar el nombre de las columnas
address_split.columns = ['nombre', 'direccion', 'ciudad', 'cod.postal']

In [35]:
address = address[['gmap_id']].join(address_split)
address.head()

,gmap_id,nombre,direccion,ciudad,cod.postal
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Porter Pharmacy,129 N Second St,Cochran,GA 31014
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,City Textile,3001 E Pico Blvd,Los Angeles,CA 90023
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,761 S Vermont Ave,Los Angeles,CA 90005
3,0x80c2c89923b27a41:0x32041559418d447,Nova Fabrics,2200 E 11th St,Los Angeles,CA 90021
4,0x80c2c632f933b073:0xc31785961fe826a6,Nobel Textile Co,719 E 9th St,Los Angeles,CA 90021


In [36]:
# Agregar una columna nueva llamada estados.
address['estado'] = None

## me di cuenta que el cod.postal contiene las letras del estado, ahi seria mas facil.
address.loc[address['cod.postal'].str.contains('CA', na=False), 'estado'] = 'California'
address.loc[address['cod.postal'].str.contains('NY', na=False), 'estado'] = 'Nueva York'
address.loc[address['cod.postal'].str.contains('IA', na=False), 'estado'] = 'Iowa'
address.loc[address['cod.postal'].str.contains('GA', na=False), 'estado'] = 'Georgia'
address.loc[address['cod.postal'].str.contains('FL', na=False), 'estado'] = 'Florida'
address.loc[address['cod.postal'].str.contains('TX', na=False), 'estado'] = 'Texas'


In [48]:
estado_none = address[address['estado'].isnull()]
estado_none

,gmap_id,nombre,direccion,ciudad,cod.postal,estado
9,0x87ee85ac3cb23b4d:0x4cd1435fddc046b1,None,None,None,None,None
25,0x86312d61f5ac28b5:0x5e6c55690b65096a,KeyMe Locksmiths,6652 Youree Dr,Shreveport,LA 71105,None
26,0x863050a2977a797d:0x8206eda0d4f6207,Fontenot Home Inspection,799 Wafer Rd,Saline,LA 71070,None
27,0x8636d2cbbf059b25:0xfb596a49eb5ae043,Basinger Family Dentistry: Brian Basinger,DDS,818 Pierremont Rd,"Shreveport, LA 71106",None
28,0x8636cd31728dc371:0x634bad3dae8adce6,Caddo Parish Assessor's Office,501 Texas St,Shreveport,LA 71101,None
...,...,...,...,...,...,...
3025001,0x54c0e365e43253f9:0x4366520651be08bb,Fuel-N-Go,33100 Van Duyn Rd,Eugene,OR 97408,None
3025003,0x54c50fa3d51bf921:0x74e65ce06582bb7e,Tyee Campground,Oregon,None,None,None
3025004,0x54950d2082e19b17:0xf68c5d099df03c8b,New Shoes Shoe Repair,12255 SW Main St #6221,Tigard,OR 97223,None
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,Steak 48,260 S Broad St,Philadelphia,PA 19102,None
